# Koppeling tussen was-wordt en definitietabel

Vinden we alle waarden voor rVvN, VvN en SBB uit de was-wordt lijst terug in de definitietabel? Alle vegetatietypen die niet automatisch tot H0000 leiden in de was wordt, zouden we willen terugvinden in de definietabel habitattypen.

Om deze twee tabellen goed te matchen, moeten we in ieder geval de volgende dingen doen:

- vegetatiecodes ontdoen van hoofdletters
- vegetatiecodes ontdoen van `leading zero's`
- VvN codes uit de was-wordt ontdubbelen
- VvN codes uit de was-wordt ontdoen van haakjes
- Codes voor rompgemeenschappen en derivaatgemeenschappen in de definitietabel ontdoen van extra formatting
- De kolom `Code vegetatietype` uit de definitietabel splitsen in twee kolommen voor VvN en SBB. 

In [74]:
import re
import pandas as pd

In [75]:
ww = pd.read_excel('../data/ww_split.xlsx')

In [76]:
ww_not_H0000 = ww.__deepcopy__()

# Verwijderen van alle rijen uit de waswordt lijst die vertalen naar H0000. Deze verwacht je niet terug te vinden in de definitietabel
ww_not_H0000 = ww_not_H0000[ww_not_H0000['Habtype']!='H0000']
ww_not_H0000 = ww_not_H0000[~ww_not_H0000['Habtype'].isna()]

# Verwijderen van alle rijen in de waswordt lijst waarin voor het habtype verwezen wordt naar subassociaties
ww_not_H0000 = ww_not_H0000[~ww_not_H0000['Habtype'].apply(lambda x: 'zie subassociaties' in x if x is not None else False)]

# Opschonen van lege waardes naar None values
ww_not_H0000['SBB'][ww_not_H0000['SBB'].isna()] = None
ww_not_H0000['VvN'][ww_not_H0000['VvN'].isna()] = None

In [77]:
dt = pd.read_excel('../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls', sheet_name=1)

In [78]:
# Verwijderen van kolommen zonder vegetatiecode. Dit zijn kolommen met waterplantengemeenschappen en vegetatieloze regels.
dt = dt[~dt['Code vegetatietype'].isna()]

# Opsplitsen van kolom `Code vegetatietype` aparte kolommen voor SBB- en VvN-code
dt['SBB'] = dt['Code vegetatietype']
dt = dt.rename(columns={'Code vegetatietype':'VvN'})
dt['VvN'][dt['VvN'].apply(lambda x: 'SBB' in x)] = None
dt['SBB'][~dt['SBB'].apply(lambda x: 'SBB' in x)] = None

# Verwijderen van leading 'SBB-' in SBB-code.
dt['SBB'] = dt['SBB'].apply(lambda x: re.sub('SBB-', '', x) if x is not None else None)

In [79]:
def disect_vvn(vvn_code: str):
    # klasse
    klasse = None
    try:
        klasse_span = re.search(r'^\d+', vvn_code).span()
        klasse = vvn_code[klasse_span[0]:klasse_span[1]]
    except:
        print('klasse in VvN code not well defined')
    
    # orde en verbond
    orde = None
    try:
        orde = re.findall('[a-zA-Z]+', vvn_code)[0]
    except:
        print('orde in VvN code not well defined')
    
    # associatie
    associatie = None
    try:
        associatie = re.findall('\d+', vvn_code)[1]
    except:
        print('associatie in VvN code not well defined')
    
    # subassociatie
    subassociatie = None
    try:
        subassociatie = re.findall('[a-zA-Z]+', vvn_code)[1]
    except:
        print('subassociatie in VvN code not well defined')
    
    return klasse, orde, associatie, subassociatie


def clean_vegcode(vegcode: str):
    k, o, a, s = disect_vvn(vegcode)

    # verwijder leading zeros uit klasse 
    k = str(int(k))
    
    # orde en verbond naar lowercase
    o = o.lower()
    
    # verwijder leading zeros uit associatie
    if a is None:
        a = ''
    else:
        a = str(int(a))
    
    # als subassociatie niet aanwezig is, maak er een lege string van.
    if s is None:
        s = ''
    s = s.lower()
        
    return k + o + a + s


def convert_dg_rg_codes(vegcode: str):
    '''
    Alleen gebruikt voor de definitietabel habitattypen. Zet codes voor Rompgemeenschappen en Derivaatgemeenschappen om in normale vegcodes, zodat de waardes matchen met die in de waswordt lijst.
    '''
    return re.sub('-', '', re.sub('\[[\d\w]*\]', '', vegcode)) if vegcode is not None else None

    
def transform_vegcode(x):
    try:
        return clean_vegcode(x)
    except:
        return None

In [80]:
ww_not_H0000['VvN'] = ww_not_H0000['VvN'].apply(lambda x: transform_vegcode(x))
dt['VvN'] = dt['VvN'].apply(lambda x: transform_vegcode(x))

klasse in VvN code not well defined
orde in VvN code not well defined
associatie in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie i

In [81]:
ww_not_H0000['SBB'] = ww_not_H0000['SBB'].apply(lambda x: x.lower() if x is not None else None)
dt['SBB'] = dt['SBB'].apply(lambda x: x.lower() if x is not None else None)

In [82]:
dt['VvN'] = dt['VvN'].apply(lambda x: convert_dg_rg_codes(x))

#### Overlap in VvN codes

In [83]:
print('{} unieke VvN codes in de was-wordt'.format(len(set(ww_not_H0000['VvN']))))
print('{} unieke VvN codes in de definitietabel'.format(len(set(dt['VvN']))))
print('{} VvN codes uit de was-wordt worden teruggevonden in de definitietabel'.format(len(set(ww_not_H0000['VvN']).intersection(set(dt['VvN'])))))

438 unieke VvN codes in de was-wordt
290 unieke VvN codes in de definitietabel
240 VvN codes uit de was-wordt worden teruggevonden in de definitietabel


In [84]:
ww_not_H0000['VvN_in_dt'] = ww_not_H0000['VvN'].apply(lambda x: x in dt['VvN'].to_list() if x is not None else None)
dt['VvN_in_ww'] = dt['VvN'].apply(lambda x: x in ww_not_H0000['VvN'].to_list() if x is not None else None)

#### Overlap SBB en definitietabel
Alle (op één na) waardes voor SBB uit de definitietabel worden teruggevonden in de waswordt. andersom is dat niet het geval, omdat alleen SBB-codes in de definitietabel staan waarvoor geen VvN code bestaat.

In [85]:
print('{} unieke SBB codes in de was-wordt'.format(len(set(ww_not_H0000['SBB']))))
print('{} unieke SBB codes in de definitietabel'.format(len(set(dt['SBB']))))
print('{} SBB codes uit de was-wordt worden teruggevonden in de definitietabel'.format(len(set(ww_not_H0000['SBB']).intersection(set(dt['SBB'])))))

549 unieke SBB codes in de was-wordt
67 unieke SBB codes in de definitietabel
66 SBB codes uit de was-wordt worden teruggevonden in de definitietabel


In [86]:
set(dt['SBB']) - (set(ww_not_H0000['SBB']).intersection(set(dt['SBB'])))

{'26-xxx [08-f]'}

In [87]:
ww_not_H0000['SBB_in_dt'] = ww_not_H0000['SBB'].apply(lambda x: x in dt['SBB'].to_list() if x is not None else None)
dt['SBB_in_ww'] = dt['SBB'].apply(lambda x: x in ww_not_H0000['SBB'].to_list() if x is not None else None)

In [89]:
ww_not_H0000.to_excel('../data/ww_cleaned.xlsx', index=False)
dt.to_excel('../data/dt_cleaned.xlsx', index=False)